In [126]:
import pandas as pd
import psycopg2
import sqlalchemy
import numpy as np
from sqlalchemy import create_engine
from scipy import stats
pd.set_option('display.max_columns', None)

In [444]:
POSTGRES_ADDRESS = 'mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com' 
POSTGRES_PORT = '5432' 
POSTGRES_USERNAME = 'lhl_student' 
POSTGRES_PASSWORD = 'lhl_student' 
POSTGRES_DBNAME='mid_term_project'
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME, 
                                                                                        password=POSTGRES_PASSWORD, 
                                                                                        ipaddress=POSTGRES_ADDRESS, 
                                                                                        port=POSTGRES_PORT, 
                                                                                        dbname=POSTGRES_DBNAME)) 
# Create the connection 
cnx = create_engine(postgres_str)

In [542]:
df_flight=pd.read_sql_query('''SELECT * FROM flights ORDER BY RANDOM() LIMIT 50000;''', cnx)

In [679]:
df_flights=df_flight.copy(deep=True)

In [680]:
df_flights=df_flights[['fl_date', 'air_time','mkt_unique_carrier', 'branded_code_share', 'mkt_carrier','mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num', 
              'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name', 'dest_airport_id', 'dest', 'dest_city_name', 
              'crs_dep_time','dep_delay','crs_arr_time', 'dup', 'crs_elapsed_time', 'flights', 'distance','arr_delay','cancelled','diverted']]

In [681]:
df_flights.columns

Index(['fl_date', 'air_time', 'mkt_unique_carrier', 'branded_code_share',
       'mkt_carrier', 'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time',
       'dep_delay', 'crs_arr_time', 'dup', 'crs_elapsed_time', 'flights',
       'distance', 'arr_delay', 'cancelled', 'diverted'],
      dtype='object')

In [682]:
df_flights.isnull().sum()

fl_date                 0
air_time              996
mkt_unique_carrier      0
branded_code_share      0
mkt_carrier             0
mkt_carrier_fl_num      0
op_unique_carrier       0
tail_num              144
op_carrier_fl_num       0
origin_airport_id       0
origin                  0
origin_city_name        0
dest_airport_id         0
dest                    0
dest_city_name          0
crs_dep_time            0
dep_delay             852
crs_arr_time            0
dup                     0
crs_elapsed_time        0
flights                 0
distance                0
arr_delay             988
cancelled               0
diverted                0
dtype: int64

In [683]:
#Dealing with missing values

"""carr_delay_mean=df_flights['carrier_delay'].astype('float').mean(axis=0)
wea_delay_mean=df_flights['weather_delay'].astype('float').mean(axis=0)
nas_delay_mean=df_flights['nas_delay'].astype('float').mean(axis=0)
sec_delay_mean=df_flights['security_delay'].astype('float').mean(axis=0)
aircraft_delay_mean=df_flights['late_aircraft_delay'].astype('float').mean(axis=0)


df_flights['carrier_delay'].replace(np.nan, carr_delay_mean, inplace=True)
df_flights['weather_delay'].replace(np.nan, wea_delay_mean, inplace=True)
df_flights['nas_delay'].replace(np.nan, nas_delay_mean, inplace=True)
df_flights['security_delay'].replace(np.nan, sec_delay_mean, inplace=True)
df_flights['late_aircraft_delay'].replace(np.nan, aircraft_delay_mean, inplace=True)"""



"carr_delay_mean=df_flights['carrier_delay'].astype('float').mean(axis=0)\nwea_delay_mean=df_flights['weather_delay'].astype('float').mean(axis=0)\nnas_delay_mean=df_flights['nas_delay'].astype('float').mean(axis=0)\nsec_delay_mean=df_flights['security_delay'].astype('float').mean(axis=0)\naircraft_delay_mean=df_flights['late_aircraft_delay'].astype('float').mean(axis=0)\n\n\ndf_flights['carrier_delay'].replace(np.nan, carr_delay_mean, inplace=True)\ndf_flights['weather_delay'].replace(np.nan, wea_delay_mean, inplace=True)\ndf_flights['nas_delay'].replace(np.nan, nas_delay_mean, inplace=True)\ndf_flights['security_delay'].replace(np.nan, sec_delay_mean, inplace=True)\ndf_flights['late_aircraft_delay'].replace(np.nan, aircraft_delay_mean, inplace=True)"

In [684]:
df_flights=df_flights.dropna()

# Feature Engineering

In [685]:
df_flights.head()

,fl_date,air_time,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_delay,crs_arr_time,dup,crs_elapsed_time,flights,distance,arr_delay,cancelled,diverted
0,2019-02-04,102.0,WN,WN,WN,701,WN,N282WN,701,14100,PHL,"Philadelphia, PA",13232,MDW,"Chicago, IL",1610,-5.0,1730,N,140.0,1.0,668.0,-19.0,0.0,0.0
1,2018-06-15,121.0,UA,UA,UA,434,UA,N38454,434,11618,EWR,"Newark, NJ",13204,MCO,"Orlando, FL",1655,60.0,1943,N,168.0,1.0,937.0,48.0,0.0,0.0
2,2019-08-19,31.0,AA,AA_CODESHARE,AA,3525,MQ,N613AE,3525,10685,BMI,"Bloomington/Normal, IL",13930,ORD,"Chicago, IL",1006,-4.0,1104,N,58.0,1.0,116.0,-4.0,0.0,0.0
3,2019-02-19,127.0,WN,WN,WN,373,WN,N8619F,373,11697,FLL,"Fort Lauderdale, FL",14100,PHL,"Philadelphia, PA",1930,8.0,2215,N,165.0,1.0,992.0,-15.0,0.0,0.0
4,2018-01-26,148.0,WN,WN,WN,1262,WN,N8327A,1262,14307,PVD,"Providence, RI",13204,MCO,"Orlando, FL",900,-2.0,1205,N,185.0,1.0,1072.0,-19.0,0.0,0.0


In [686]:
# Converting date and time columns
df_flights['fl_date'] = pd.to_datetime(df_flights['fl_date'], errors='coerce')
df_flights['month'] = df_flights['fl_date'].dt.month
df_flights['day_of_week'] = df_flights['fl_date'].dt.dayofweek
df_flights['day_of_month'] = df_flights['fl_date'].dt.day
df_flights['year'] = df_flights['fl_date'].dt.year

In [687]:
# Creating columns for departure and arrival hours
df_flights['crs_dep_time'] = df_flights['crs_dep_time'].apply(lambda x: '{0:0>4}'.format(x))
df_flights['crs_arr_time'] = df_flights['crs_arr_time'].apply(lambda x: '{0:0>4}'.format(x))
df_flights['dep_hour']=df_flights['crs_dep_time'].str[:2]
df_flights['arr_hour']=df_flights['crs_arr_time'].str[:2]

In [688]:
# Dividing the months into four seasons
df_flights['seasons']='spring'
df_flights.loc[df_flights['month'].between(6, 8), 'seasons']= 'summer'
df_flights.loc[df_flights['month'].between(9, 11), 'seasons']= 'autum'
df_flights.loc[df_flights['month'].between(12, 2), 'seasons']= 'winter'

In [689]:
df_flights.head()

,fl_date,air_time,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_delay,crs_arr_time,dup,crs_elapsed_time,flights,distance,arr_delay,cancelled,diverted,month,day_of_week,day_of_month,year,dep_hour,arr_hour,seasons
0,2019-02-04,102.0,WN,WN,WN,701,WN,N282WN,701,14100,PHL,"Philadelphia, PA",13232,MDW,"Chicago, IL",1610,-5.0,1730,N,140.0,1.0,668.0,-19.0,0.0,0.0,2,0,4,2019,16,17,spring
1,2018-06-15,121.0,UA,UA,UA,434,UA,N38454,434,11618,EWR,"Newark, NJ",13204,MCO,"Orlando, FL",1655,60.0,1943,N,168.0,1.0,937.0,48.0,0.0,0.0,6,4,15,2018,16,19,summer
2,2019-08-19,31.0,AA,AA_CODESHARE,AA,3525,MQ,N613AE,3525,10685,BMI,"Bloomington/Normal, IL",13930,ORD,"Chicago, IL",1006,-4.0,1104,N,58.0,1.0,116.0,-4.0,0.0,0.0,8,0,19,2019,10,11,summer
3,2019-02-19,127.0,WN,WN,WN,373,WN,N8619F,373,11697,FLL,"Fort Lauderdale, FL",14100,PHL,"Philadelphia, PA",1930,8.0,2215,N,165.0,1.0,992.0,-15.0,0.0,0.0,2,1,19,2019,19,22,spring
4,2018-01-26,148.0,WN,WN,WN,1262,WN,N8327A,1262,14307,PVD,"Providence, RI",13204,MCO,"Orlando, FL",0900,-2.0,1205,N,185.0,1.0,1072.0,-19.0,0.0,0.0,1,4,26,2018,09,12,spring


In [690]:
# Dividing the depature and arrival hours into day sections
df_flights['dep_hour']=df_flights['dep_hour'].astype(int)
df_flights['dep_time_during_day']='night'
df_flights.loc[df_flights['dep_hour'].between(12,18), 'dep_time_during_day']= 'afternoon'
df_flights.loc[df_flights['dep_hour'].between(19,23), 'dep_time_during_day']= 'evening'
df_flights.loc[df_flights['dep_hour'].between(4,11), 'dep_time_during_day']= 'morning'

df_flights['arr_hour']=df_flights['arr_hour'].astype(int)
df_flights['arr_time_during_day']='night'
df_flights.loc[df_flights['arr_hour'].between(12,18), 'arr_time_during_day']= 'afternoon'
df_flights.loc[df_flights['arr_hour'].between(19,23), 'arr_time_during_day']= 'evening'
df_flights.loc[df_flights['arr_hour'].between(4,11), 'arr_time_during_day']= 'morning'

In [691]:
df_flights.head()

,fl_date,air_time,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_delay,crs_arr_time,dup,crs_elapsed_time,flights,distance,arr_delay,cancelled,diverted,month,day_of_week,day_of_month,year,dep_hour,arr_hour,seasons,dep_time_during_day,arr_time_during_day
0,2019-02-04,102.0,WN,WN,WN,701,WN,N282WN,701,14100,PHL,"Philadelphia, PA",13232,MDW,"Chicago, IL",1610,-5.0,1730,N,140.0,1.0,668.0,-19.0,0.0,0.0,2,0,4,2019,16,17,spring,afternoon,afternoon
1,2018-06-15,121.0,UA,UA,UA,434,UA,N38454,434,11618,EWR,"Newark, NJ",13204,MCO,"Orlando, FL",1655,60.0,1943,N,168.0,1.0,937.0,48.0,0.0,0.0,6,4,15,2018,16,19,summer,afternoon,evening
2,2019-08-19,31.0,AA,AA_CODESHARE,AA,3525,MQ,N613AE,3525,10685,BMI,"Bloomington/Normal, IL",13930,ORD,"Chicago, IL",1006,-4.0,1104,N,58.0,1.0,116.0,-4.0,0.0,0.0,8,0,19,2019,10,11,summer,morning,morning
3,2019-02-19,127.0,WN,WN,WN,373,WN,N8619F,373,11697,FLL,"Fort Lauderdale, FL",14100,PHL,"Philadelphia, PA",1930,8.0,2215,N,165.0,1.0,992.0,-15.0,0.0,0.0,2,1,19,2019,19,22,spring,evening,evening
4,2018-01-26,148.0,WN,WN,WN,1262,WN,N8327A,1262,14307,PVD,"Providence, RI",13204,MCO,"Orlando, FL",0900,-2.0,1205,N,185.0,1.0,1072.0,-19.0,0.0,0.0,1,4,26,2018,9,12,spring,morning,afternoon


In [692]:
df_flights = df_flights.drop('fl_date', axis = 1)

In [693]:
df_flights['dep_time_during_day'].value_counts()

morning      20435
afternoon    20385
evening       8000
night          162
Name: dep_time_during_day, dtype: int64

In [694]:
#Converting to air_time and arr_delay to category
def flight_duration(x):
    if x <=180:
        return 'Short'
    elif x >180 and x<360:
        return 'Medium'
    elif x>360:
        return 'Long'

df_flights['flight_duration_type']=df_flights['air_time'].apply(lambda x: flight_duration(x))
df_flights['flight_duration_type'].value_counts()

Short     42431
Medium     6374
Long        171
Name: flight_duration_type, dtype: int64

In [695]:
# Separating State name from city
df_flights['origin_state']=df_flights['origin_city_name'].apply(lambda x: x.split(', ')[1])
df_flights['destination_state']=df_flights['dest_city_name'].apply(lambda x: x.split(', ')[1])

In [696]:
# Finding the mean of departure and arrival delays based on different carriers
df_flights['dep_delay_mean_by_carrier']=df_flights.groupby(['op_unique_carrier'])['dep_delay'].transform(np.mean)
df_flights['arr_delay_mean_by_carrier']=df_flights.groupby(['op_unique_carrier'])['arr_delay'].transform(np.mean)

In [697]:
df_flights.columns

Index(['air_time', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time',
       'dep_delay', 'crs_arr_time', 'dup', 'crs_elapsed_time', 'flights',
       'distance', 'arr_delay', 'cancelled', 'diverted', 'month',
       'day_of_week', 'day_of_month', 'year', 'dep_hour', 'arr_hour',
       'seasons', 'dep_time_during_day', 'arr_time_during_day',
       'flight_duration_type', 'origin_state', 'destination_state',
       'dep_delay_mean_by_carrier', 'arr_delay_mean_by_carrier'],
      dtype='object')

In [698]:
# Finding the mean of different delay types based on different carriers
"""df_flights['weather_delay_mean_by_carrier']=df_flights.groupby(['op_unique_carrier'])['weather_delay'].transform(np.mean)
df_flights['security_delay_mean_by_carrier']=df_flights.groupby(['op_unique_carrier'])['security_delay'].transform(np.mean)
df_flights['carrier_delay_mean_by_carrier']=df_flights.groupby(['op_unique_carrier'])['carrier_delay'].transform(np.mean)
df_flights['late_aircraft_delay_mean_by_carrier']=df_flights.groupby(['op_unique_carrier'])['late_aircraft_delay'].transform(np.mean)
df_flights['nas_delay_mean_by_carrier']=df_flights.groupby(['op_unique_carrier'])['nas_delay'].transform(np.mean)"""

"df_flights['weather_delay_mean_by_carrier']=df_flights.groupby(['op_unique_carrier'])['weather_delay'].transform(np.mean)\ndf_flights['security_delay_mean_by_carrier']=df_flights.groupby(['op_unique_carrier'])['security_delay'].transform(np.mean)\ndf_flights['carrier_delay_mean_by_carrier']=df_flights.groupby(['op_unique_carrier'])['carrier_delay'].transform(np.mean)\ndf_flights['late_aircraft_delay_mean_by_carrier']=df_flights.groupby(['op_unique_carrier'])['late_aircraft_delay'].transform(np.mean)\ndf_flights['nas_delay_mean_by_carrier']=df_flights.groupby(['op_unique_carrier'])['nas_delay'].transform(np.mean)"

In [699]:
#make the types categories
"""df_flights["mkt_carrier"] = df_flights["mkt_carrier"].astype("category")
df_flights["op_unique_carrier"] = df_flights["op_unique_carrier"].astype("category")
df_flights["tail_num"] = df_flights["tail_num"].astype("category")
df_flights["op_carrier_fl_num"] = df_flights["op_carrier_fl_num"].astype("category")
df_flights["origin_airport_id"] = df_flights["origin_airport_id"].astype("category")
df_flights["dest_airport_id"] = df_flights["dest_airport_id"].astype("category")
df_flights["mkt_carrier_fl_num"] = df_flights["mkt_carrier_fl_num"].astype("category")"""

'df_flights["mkt_carrier"] = df_flights["mkt_carrier"].astype("category")\ndf_flights["op_unique_carrier"] = df_flights["op_unique_carrier"].astype("category")\ndf_flights["tail_num"] = df_flights["tail_num"].astype("category")\ndf_flights["op_carrier_fl_num"] = df_flights["op_carrier_fl_num"].astype("category")\ndf_flights["origin_airport_id"] = df_flights["origin_airport_id"].astype("category")\ndf_flights["dest_airport_id"] = df_flights["dest_airport_id"].astype("category")\ndf_flights["mkt_carrier_fl_num"] = df_flights["mkt_carrier_fl_num"].astype("category")'

In [701]:
#encode arrival delay
df_flights['arr_delay_cat'] = df_flights['arr_delay'].apply(lambda x: 1 if x > 0 else 0)
# ENCODE AIRPORTS AND TAILNUM
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
df_flights['mkt_carrier'] = encoder.fit_transform(df_flights[['mkt_carrier']])
df_flights['mkt_unique_carrier'] = encoder.fit_transform(df_flights[['mkt_unique_carrier']])
df_flights['mkt_carrier_fl_num'] = encoder.fit_transform(df_flights[['mkt_carrier_fl_num']])
df_flights['op_unique_carrier'] = encoder.fit_transform(df_flights[['op_unique_carrier']])
df_flights['tail_num'] = encoder.fit_transform(df_flights[['tail_num']])
df_flights['op_carrier_fl_num'] = encoder.fit_transform(df_flights[['op_carrier_fl_num']])
df_flights['origin_airport_id'] = encoder.fit_transform(df_flights[['origin_airport_id']])
df_flights['dest_airport_id'] = encoder.fit_transform(df_flights[['dest_airport_id']])

In [702]:
# Finding the air traffics based on the airports
df_flights['origin_airport_taffic']=df_flights.groupby(['origin_airport_id'])['flights'].transform(np.sum)
df_flights['dest_airport_traffic']=df_flights.groupby(['dest_airport_id'])['flights'].transform(np.sum)

In [703]:
"""# Finding the arrival and departure delays of airports per months
df_flights['total_arr_delay_dest_aiport_month']=df_flights.groupby(['dest_airport_id', 'month'])['arr_delay'].transform(np.sum)
df_flights['total_dep_delay_dest_aiport_month']=df_flights.groupby(['dest_airport_id', 'month'])['dep_delay'].transform(np.sum)

df_flights['total_arr_delay_origin_aiport_month']=df_flights.groupby(['origin_airport_id', 'month'])['arr_delay'].transform(np.sum)
df_flights['total_dep_delay_origin_aiport_month']=df_flights.groupby(['origin_airport_id', 'month'])['dep_delay'].transform(np.sum)
"""


"# Finding the arrival and departure delays of airports per months\ndf_flights['total_arr_delay_dest_aiport_month']=df_flights.groupby(['dest_airport_id', 'month'])['arr_delay'].transform(np.sum)\ndf_flights['total_dep_delay_dest_aiport_month']=df_flights.groupby(['dest_airport_id', 'month'])['dep_delay'].transform(np.sum)\n\ndf_flights['total_arr_delay_origin_aiport_month']=df_flights.groupby(['origin_airport_id', 'month'])['arr_delay'].transform(np.sum)\ndf_flights['total_dep_delay_origin_aiport_month']=df_flights.groupby(['origin_airport_id', 'month'])['dep_delay'].transform(np.sum)\n"

In [704]:
"""# Finding the arrival and departure delays of airports per day
df_flights['total_arr_delay_dest_aiport_day']=df_flights.groupby(['dest_airport_id', 'day_of_month'])['arr_delay'].transform(np.sum)
df_flights['total_dep_delay_dest_aiport_day']=df_flights.groupby(['dest_airport_id', 'day_of_month'])['dep_delay'].transform(np.sum)

df_flights['total_arr_delay_origin_aiport_day']=df_flights.groupby(['origin_airport_id', 'day_of_month'])['arr_delay'].transform(np.sum)
df_flights['total_dep_delay_origin_aiport_day']=df_flights.groupby(['origin_airport_id', 'day_of_month'])['dep_delay'].transform(np.sum)
"""



"# Finding the arrival and departure delays of airports per day\ndf_flights['total_arr_delay_dest_aiport_day']=df_flights.groupby(['dest_airport_id', 'day_of_month'])['arr_delay'].transform(np.sum)\ndf_flights['total_dep_delay_dest_aiport_day']=df_flights.groupby(['dest_airport_id', 'day_of_month'])['dep_delay'].transform(np.sum)\n\ndf_flights['total_arr_delay_origin_aiport_day']=df_flights.groupby(['origin_airport_id', 'day_of_month'])['arr_delay'].transform(np.sum)\ndf_flights['total_dep_delay_origin_aiport_day']=df_flights.groupby(['origin_airport_id', 'day_of_month'])['dep_delay'].transform(np.sum)\n"

In [705]:
"""# Finding the arrival and departure delays of airports per year
df_flights['total_arr_delay_dest_aiport_year']=df_flights.groupby(['dest_airport_id', 'year'])['arr_delay'].transform(np.sum)
df_flights['total_dep_delay_dest_aiport_year']=df_flights.groupby(['dest_airport_id', 'year'])['dep_delay'].transform(np.sum)

df_flights['total_arr_delay_origin_aiport_year']=df_flights.groupby(['origin_airport_id', 'year'])['arr_delay'].transform(np.sum)
df_flights['total_dep_delay_origin_aiport_year']=df_flights.groupby(['origin_airport_id', 'year'])['dep_delay'].transform(np.sum)
"""




"# Finding the arrival and departure delays of airports per year\ndf_flights['total_arr_delay_dest_aiport_year']=df_flights.groupby(['dest_airport_id', 'year'])['arr_delay'].transform(np.sum)\ndf_flights['total_dep_delay_dest_aiport_year']=df_flights.groupby(['dest_airport_id', 'year'])['dep_delay'].transform(np.sum)\n\ndf_flights['total_arr_delay_origin_aiport_year']=df_flights.groupby(['origin_airport_id', 'year'])['arr_delay'].transform(np.sum)\ndf_flights['total_dep_delay_origin_aiport_year']=df_flights.groupby(['origin_airport_id', 'year'])['dep_delay'].transform(np.sum)\n"

In [706]:
df_flights.isnull().sum()

air_time                     0
mkt_unique_carrier           0
branded_code_share           0
mkt_carrier                  0
mkt_carrier_fl_num           0
op_unique_carrier            0
tail_num                     0
op_carrier_fl_num            0
origin_airport_id            0
origin                       0
origin_city_name             0
dest_airport_id              0
dest                         0
dest_city_name               0
crs_dep_time                 0
dep_delay                    0
crs_arr_time                 0
dup                          0
crs_elapsed_time             0
flights                      0
distance                     0
arr_delay                    0
cancelled                    0
diverted                     0
month                        0
day_of_week                  0
day_of_month                 0
year                         0
dep_hour                     0
arr_hour                     0
seasons                      0
dep_time_during_day          0
arr_time

In [707]:
# Converting categorical data by getting dummies


dummy_durtypes=pd.get_dummies(df_flights[['flight_duration_type', 'dep_time_during_day', 'arr_time_during_day', 'seasons']])

df_flights=pd.concat([df_flights, dummy_durtypes], axis=1).drop(['flight_duration_type', 'dep_time_during_day', 'arr_time_during_day', 'seasons'], axis=1)

In [708]:
df_flights=df_flights.dropna()

In [709]:
df_flights.head()

,air_time,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_delay,crs_arr_time,dup,crs_elapsed_time,flights,distance,arr_delay,cancelled,diverted,month,day_of_week,day_of_month,year,dep_hour,arr_hour,origin_state,destination_state,dep_delay_mean_by_carrier,arr_delay_mean_by_carrier,arr_delay_cat,origin_airport_taffic,dest_airport_traffic,flight_duration_type_Long,flight_duration_type_Medium,flight_duration_type_Short,dep_time_during_day_afternoon,dep_time_during_day_evening,dep_time_during_day_morning,dep_time_during_day_night,arr_time_during_day_afternoon,arr_time_during_day_evening,arr_time_during_day_morning,arr_time_during_day_night,seasons_autum,seasons_spring,seasons_summer
0,102.0,10.0,WN,10.0,698.0,24.0,1179.0,698.0,263.0,PHL,"Philadelphia, PA",217.0,MDW,"Chicago, IL",1610,-5.0,1730,N,140.0,1.0,668.0,-19.0,0.0,0.0,2,0,4,2019,16,17,PA,IL,11.015193,4.026319,0,960.0,512.0,0,0,1,1,0,0,0,1,0,0,0,0,1,0
1,121.0,8.0,UA,8.0,431.0,22.0,1822.0,431.0,112.0,EWR,"Newark, NJ",215.0,MCO,"Orlando, FL",1655,60.0,1943,N,168.0,1.0,937.0,48.0,0.0,0.0,6,4,15,2018,16,19,NJ,FL,11.679886,7.145671,1,955.0,790.0,0,0,1,1,0,0,0,0,1,0,0,0,0,1
2,31.0,0.0,AA_CODESHARE,0.0,3489.0,16.0,2933.0,3489.0,40.0,BMI,"Bloomington/Normal, IL",252.0,ORD,"Chicago, IL",1006,-4.0,1104,N,58.0,1.0,116.0,-4.0,0.0,0.0,8,0,19,2019,10,11,IL,IL,8.847656,6.121094,0,16.0,2400.0,0,0,1,0,0,1,0,0,0,1,0,0,0,1
3,127.0,10.0,WN,10.0,370.0,24.0,4667.0,370.0,120.0,FLL,"Fort Lauderdale, FL",265.0,PHL,"Philadelphia, PA",1930,8.0,2215,N,165.0,1.0,992.0,-15.0,0.0,0.0,2,1,19,2019,19,22,FL,PA,11.015193,4.026319,0,652.0,872.0,0,0,1,0,1,0,0,0,1,0,0,0,1,0
4,148.0,10.0,WN,10.0,1257.0,24.0,4373.0,1257.0,281.0,PVD,"Providence, RI",215.0,MCO,"Orlando, FL",0900,-2.0,1205,N,185.0,1.0,1072.0,-19.0,0.0,0.0,1,4,26,2018,9,12,RI,FL,11.015193,4.026319,0,123.0,790.0,0,0,1,0,0,1,0,1,0,0,0,0,1,0


In [710]:
df_flights=df_flights.drop(['dep_delay','mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
                            'mkt_carrier_fl_num', 'op_unique_carrier','op_carrier_fl_num', 
                            'origin', 'origin_city_name','dup','dest','dest_city_name',
                            'origin_state', 'destination_state', 
                            'cancelled','diverted',
                           ], axis=1)

In [711]:
df_flights

,air_time,tail_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,flights,distance,arr_delay,month,day_of_week,day_of_month,year,dep_hour,arr_hour,dep_delay_mean_by_carrier,arr_delay_mean_by_carrier,arr_delay_cat,origin_airport_taffic,dest_airport_traffic,flight_duration_type_Long,flight_duration_type_Medium,flight_duration_type_Short,dep_time_during_day_afternoon,dep_time_during_day_evening,dep_time_during_day_morning,dep_time_during_day_night,arr_time_during_day_afternoon,arr_time_during_day_evening,arr_time_during_day_morning,arr_time_during_day_night,seasons_autum,seasons_spring,seasons_summer
0,102.0,1179.0,263.0,217.0,1610,1730,140.0,1.0,668.0,-19.0,2,0,4,2019,16,17,11.015193,4.026319,0,960.0,512.0,0,0,1,1,0,0,0,1,0,0,0,0,1,0
1,121.0,1822.0,112.0,215.0,1655,1943,168.0,1.0,937.0,48.0,6,4,15,2018,16,19,11.679886,7.145671,1,955.0,790.0,0,0,1,1,0,0,0,0,1,0,0,0,0,1
2,31.0,2933.0,40.0,252.0,1006,1104,58.0,1.0,116.0,-4.0,8,0,19,2019,10,11,8.847656,6.121094,0,16.0,2400.0,0,0,1,0,0,1,0,0,0,1,0,0,0,1
3,127.0,4667.0,120.0,265.0,1930,2215,165.0,1.0,992.0,-15.0,2,1,19,2019,19,22,11.015193,4.026319,0,652.0,872.0,0,0,1,0,1,0,0,0,1,0,0,0,1,0
4,148.0,4373.0,281.0,215.0,0900,1205,185.0,1.0,1072.0,-19.0,1,4,26,2018,9,12,11.015193,4.026319,0,123.0,790.0,0,0,1,0,0,1,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,168.0,5400.0,90.0,180.0,0800,1231,211.0,1.0,1391.0,-22.0,3,2,20,2019,8,12,11.472062,5.147078,0,1739.0,819.0,0,0,1,0,0,1,0,1,0,0,0,0,1,0
49996,110.0,4962.0,245.0,20.0,1548,1857,129.0,1.0,761.0,-3.0,6,6,16,2019,15,18,8.240300,0.863384,0,172.0,2498.0,0,0,1,1,0,0,0,1,0,0,0,0,0,1
49997,24.0,2373.0,97.0,27.0,2220,2307,47.0,1.0,113.0,-12.0,9,4,20,2019,22,23,10.808971,6.446135,0,1073.0,30.0,0,0,1,0,1,0,0,0,1,0,0,1,0,0
49998,175.0,528.0,312.0,92.0,1330,1924,234.0,1.0,1464.0,2.0,5,3,3,2018,13,19,10.808971,6.446135,1,1069.0,1795.0,0,0,1,1,0,0,0,0,1,0,0,0,1,0


In [719]:
df_flights.to_csv(r'cleaned_flights5.csv')

In [713]:
df_model=pd.read_csv('cleaned_flights.csv')

In [714]:
df_modeling=df_model.copy(deep=True)

In [715]:
df_modeling.columns

Index(['Unnamed: 0', 'air_time', 'tail_num', 'origin_airport_id',
       'dest_airport_id', 'crs_dep_time', 'crs_arr_time', 'crs_elapsed_time',
       'flights', 'distance', 'arr_delay', 'month', 'day_of_week',
       'day_of_month', 'year', 'dep_hour', 'arr_hour',
       'dep_delay_mean_by_carrier', 'arr_delay_mean_by_carrier',
       'arr_delay_cat', 'origin_airport_taffic', 'dest_airport_traffic',
       'flight_duration_type_Long', 'flight_duration_type_Medium',
       'flight_duration_type_Short', 'dep_time_during_day_afternoon',
       'dep_time_during_day_evening', 'dep_time_during_day_morning',
       'dep_time_during_day_night', 'arr_time_during_day_afternoon',
       'arr_time_during_day_evening', 'arr_time_during_day_morning',
       'arr_time_during_day_night', 'seasons_autum', 'seasons_spring',
       'seasons_summer'],
      dtype='object')

In [716]:
df_modeling.head()

,Unnamed: 0,air_time,tail_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,flights,distance,arr_delay,month,day_of_week,day_of_month,year,dep_hour,arr_hour,dep_delay_mean_by_carrier,arr_delay_mean_by_carrier,arr_delay_cat,origin_airport_taffic,dest_airport_traffic,flight_duration_type_Long,flight_duration_type_Medium,flight_duration_type_Short,dep_time_during_day_afternoon,dep_time_during_day_evening,dep_time_during_day_morning,dep_time_during_day_night,arr_time_during_day_afternoon,arr_time_during_day_evening,arr_time_during_day_morning,arr_time_during_day_night,seasons_autum,seasons_spring,seasons_summer
0,0,102.0,1179.0,263.0,217.0,1610,1730,140.0,1.0,668.0,-19.0,2,0,4,2019,16,17,11.015193,4.026319,0,960.0,512.0,0,0,1,1,0,0,0,1,0,0,0,0,1,0
1,1,121.0,1822.0,112.0,215.0,1655,1943,168.0,1.0,937.0,48.0,6,4,15,2018,16,19,11.679886,7.145671,1,955.0,790.0,0,0,1,1,0,0,0,0,1,0,0,0,0,1
2,2,31.0,2933.0,40.0,252.0,1006,1104,58.0,1.0,116.0,-4.0,8,0,19,2019,10,11,8.847656,6.121094,0,16.0,2400.0,0,0,1,0,0,1,0,0,0,1,0,0,0,1
3,3,127.0,4667.0,120.0,265.0,1930,2215,165.0,1.0,992.0,-15.0,2,1,19,2019,19,22,11.015193,4.026319,0,652.0,872.0,0,0,1,0,1,0,0,0,1,0,0,0,1,0
4,4,148.0,4373.0,281.0,215.0,900,1205,185.0,1.0,1072.0,-19.0,1,4,26,2018,9,12,11.015193,4.026319,0,123.0,790.0,0,0,1,0,0,1,0,1,0,0,0,0,1,0


In [717]:
df_modeling.isnull().sum()

Unnamed: 0                       0
air_time                         0
tail_num                         0
origin_airport_id                0
dest_airport_id                  0
crs_dep_time                     0
crs_arr_time                     0
crs_elapsed_time                 0
flights                          0
distance                         0
arr_delay                        0
month                            0
day_of_week                      0
day_of_month                     0
year                             0
dep_hour                         0
arr_hour                         0
dep_delay_mean_by_carrier        0
arr_delay_mean_by_carrier        0
arr_delay_cat                    0
origin_airport_taffic            0
dest_airport_traffic             0
flight_duration_type_Long        0
flight_duration_type_Medium      0
flight_duration_type_Short       0
dep_time_during_day_afternoon    0
dep_time_during_day_evening      0
dep_time_during_day_morning      0
dep_time_during_day_

In [718]:
df_modeling=df_modeling.drop(['Unnamed: 0', 'air_time'], axis=1)

# Modeling

In [659]:
X=df_modeling.drop('arr_delay', axis=1)
y=df_modeling['arr_delay']

In [660]:
X.columns

Index(['tail_num', 'origin_airport_id', 'dest_airport_id', 'crs_dep_time',
       'crs_arr_time', 'crs_elapsed_time', 'flights', 'distance', 'month',
       'day_of_week', 'day_of_month', 'year', 'dep_hour', 'arr_hour',
       'dep_delay_mean_by_carrier', 'arr_delay_mean_by_carrier',
       'origin_airport_taffic', 'dest_airport_traffic',
       'flight_duration_type_Long', 'flight_duration_type_Medium',
       'flight_duration_type_Short', 'dep_time_during_day_afternoon',
       'dep_time_during_day_evening', 'dep_time_during_day_morning',
       'dep_time_during_day_night', 'arr_time_during_day_afternoon',
       'arr_time_during_day_evening', 'arr_time_during_day_morning',
       'arr_time_during_day_night', 'seasons_autum', 'seasons_spring',
       'seasons_summer'],
      dtype='object')

In [661]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scale = scaler.fit_transform(X)
X_scale = pd.DataFrame(X_scale,columns=X.columns)

In [662]:
# Spliting into Train and Test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test=train_test_split(X_scale, y, test_size=0.25, random_state=100, shuffle=True)

In [663]:
#Random Forest Regressors
from sklearn.ensemble import RandomForestRegressor

clf=RandomForestRegressor(n_estimators=40, max_depth=6, random_state=0)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)


In [664]:

from sklearn.metrics import mean_absolute_error
print('Mean Absolue Error: ', mean_absolute_error(y_test, y_pred))

from sklearn.metrics import mean_squared_error
print('Mean Squared Error: ', mean_squared_error(y_test, y_pred))

from sklearn.metrics import r2_score
print('R2_score: ', r2_score(y_test, y_pred))
print('RMSE: ', np.sqrt(mean_squared_error(y_test, y_pred)))

Mean Absolue Error:  24.65426804475684
Mean Squared Error:  2678.641412016271
R2_score:  0.010429780904223529
RMSE:  51.75559305057059


In [665]:
#Linear Regression
from sklearn.linear_model import LinearRegression

clf=LinearRegression()
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)

In [666]:

print('Mean Absolue Error: ', mean_absolute_error(y_test, y_pred))
print('Mean Squared Error: ', mean_squared_error(y_test, y_pred))
print('R2_score: ', r2_score(y_test, y_pred))
print('RMSE: ', np.sqrt(mean_squared_error(y_test, y_pred)))

Mean Absolue Error:  24.672876232088548
Mean Squared Error:  2667.9504775746755
R2_score:  0.014379331706450893
RMSE:  51.652206899363705


In [667]:
importance=clf.coef_
for i, v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))

Feature: 0, Score: -0.12596
Feature: 1, Score: -0.36481
Feature: 2, Score: 0.14386
Feature: 3, Score: -5.84427
Feature: 4, Score: 0.37006
Feature: 5, Score: -4.66589
Feature: 6, Score: 0.00000
Feature: 7, Score: 2.11032
Feature: 8, Score: -0.17434
Feature: 9, Score: -0.41027
Feature: 10, Score: -0.07386
Feature: 11, Score: 0.67789
Feature: 12, Score: 7.97520
Feature: 13, Score: 4.33873
Feature: 14, Score: -0.24373
Feature: 15, Score: 3.53862
Feature: 16, Score: 0.87625
Feature: 17, Score: 1.38283
Feature: 18, Score: 0.94782
Feature: 19, Score: -1.17180
Feature: 20, Score: -2.46782
Feature: 21, Score: 0.80213
Feature: 22, Score: -0.91245
Feature: 23, Score: -0.19651
Feature: 24, Score: 0.67592
Feature: 25, Score: -0.35807
Feature: 26, Score: -0.93066
Feature: 27, Score: 0.55309
Feature: 28, Score: 2.53470
Feature: 29, Score: -1.26218
Feature: 30, Score: -0.26669
Feature: 31, Score: 1.53901


In [668]:
# Naives Bayes, GaussianNB Naive Bayes
from sklearn.naive_bayes import GaussianNB

clf=GaussianNB()
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)

In [669]:

print('Mean Absolue Error: ', mean_absolute_error(y_test, y_pred))
print('Mean Squared Error: ', mean_squared_error(y_test, y_pred))
print('R2_score: ', r2_score(y_test, y_pred))
print('RMSE: ', np.sqrt(mean_squared_error(y_test, y_pred)))

Mean Absolue Error:  216.81471500898252
Mean Squared Error:  53539.04319777887
R2_score:  -18.77892317715069
RMSE:  231.38505396368814


In [670]:
# Random Forest Classifier 
from sklearn.ensemble import RandomForestClassifier

clf=RandomForestClassifier(n_estimators=50, max_depth=10,)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)

from sklearn.metrics import accuracy_score

print('Mean Squared Error: ',mean_squared_error(y_test, y_pred))
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('R2 Score: ', r2_score(y_test, y_pred))

Mean Squared Error:  3056.202188469704
Accuracy:  0.028172464478196962
R2 Score:  -0.12905245759210504


In [671]:
# Ridge Regression
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

clf=Ridge()
parameters={'alpha':[0.01,0.1,0.5,1,5]}

ridge_grid=GridSearchCV(clf, param_grid=parameters, scoring='r2', cv=10)

ridge_grid.fit(X_train, y_train)
y_pred=ridge_grid.best_estimator_.predict(X_test)

print('R2_score: ', r2_score(y_test, y_pred))
print('RMSE: ', np.sqrt(mean_squared_error(y_test, y_pred)))

R2_score:  0.014389463024027549
RMSE:  51.651941428948334
